<a href="https://colab.research.google.com/github/JulieWang417/DS-Unit-2-Linear-Models/blob/master/module4-logistic-regression/Julie_Wang_assignment_regression_classification_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [ ] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [ ] Begin with baselines for classification.
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [ ] Get your model's test accuracy. (One time, at the end.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [0]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [0]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [0]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [0]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

### Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.

In [7]:
df.head(2)

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
0,California,1/18/2016,3.5,4.2,NaN,6.49,3.0,NaN,NaN,NaN,NaN,NaN,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,California,1/24/2016,3.5,3.3,NaN,5.45,3.5,NaN,NaN,NaN,NaN,NaN,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [8]:
df.shape

(421, 59)

In [9]:
df.isnull().sum()

Burrito             0
Date                0
Yelp              334
Google            334
Chips             395
Cost                7
Hunger              3
Mass (g)          399
Density (g/mL)    399
Length            138
Circum            140
Volume            140
Tortilla            0
Temp               20
Meat               14
Fillings            3
Meat:filling        9
Uniformity          2
Salsa              25
Synergy             2
Wrap                3
Unreliable        388
NonSD             414
Beef              242
Pico              263
Guac              267
Cheese            262
Fries             294
Sour cream        329
Pork              370
Chicken           400
Shrimp            400
Fish              415
Rice              385
Beans             386
Lettuce           410
Tomato            414
Bell peper        414
Carrots           420
Cabbage           413
Sauce             383
Salsa.1           414
Cilantro          406
Onion             404
Taquito           417
Pineapple 

In [0]:
condition = df.isnull().sum() < 100

In [0]:
df = df.loc[:,condition]

In [12]:
df.isnull().sum()

Burrito          0
Date             0
Cost             7
Hunger           3
Tortilla         0
Temp            20
Meat            14
Fillings         3
Meat:filling     9
Uniformity       2
Salsa           25
Synergy          2
Wrap             3
Great            0
dtype: int64

In [0]:
df['Date']=pd.to_datetime(df['Date'])

train= df[df.Date.dt.year < 2017]
val= df[df.Date.dt.year == 2017]
test= df[df.Date.dt.year > 2017]

In [14]:
train.Date.dt.year.value_counts()

2016    296
2015      1
2011      1
Name: Date, dtype: int64

In [15]:
test.Date.dt.year.value_counts()

2018    27
2019    10
2026     1
Name: Date, dtype: int64

In [16]:
val.Date.dt.year.value_counts()

2017    85
Name: Date, dtype: int64

In [17]:
train.shape,val.shape,test.shape

((298, 14), (85, 14), (38, 14))

In [0]:
target = 'Great'
features = df.columns.drop([target,'Date'])

In [0]:
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]
X_val = test[features]
y_val = test[target]

In [20]:
X_train.head(2)

,Burrito,Cost,Hunger,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap
0,California,6.49,3.0,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0
1,California,5.45,3.5,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0


In [21]:
y_train.head()

0    False
1    False
2    False
3    False
4     True
Name: Great, dtype: bool

## Begin with baselines for classification.

In [0]:
# Encoding 
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_val= encoder.transform(X_val)
X_test= encoder.transform(X_test)

In [23]:
X_train.head()

,Burrito_California,Burrito_Carnitas,Burrito_Asada,Burrito_Other,Burrito_Surf & Turf,Cost,Hunger,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap
0,1,0,0,0,0,6.49,3.0,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0
1,1,0,0,0,0,5.45,3.5,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0
2,0,1,0,0,0,4.85,1.5,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0
3,0,0,1,0,0,5.25,2.0,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0
4,1,0,0,0,0,6.59,4.0,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0


In [0]:
majority_class = y_train.mode()[0]
y_pred = [majority_class] * len(y_train)

In [25]:
majority_class

False

In [26]:
# Training accuracy of majority class baseline = 
# frequency of majority class (aka base rate)
from sklearn.metrics import accuracy_score
a1 = accuracy_score(y_train, y_pred)
print(f'Training accuracy of majority class baseline is ${a1}.')

Training accuracy of majority class baseline is $0.5906040268456376.


In [27]:
# Validation accuracy of majority class baseline = 
# usually similar to Train accuracy

y_pred = [majority_class] * len(y_val)
a2 = accuracy_score(y_val, y_pred)
print(f'Validation accuracy of majority class baseline is ${a2}.')

Validation accuracy of majority class baseline is $0.42105263157894735.


## Use scikit-learn for logistic regression.

In [0]:
# Impute missing values
from sklearn.impute import SimpleImputer
imputer = SimpleImputer()
X_train_imputed = imputer.fit_transform(X_train)
X_val_imputed = imputer.transform(X_val)
X_test_imputed = imputer.transform(X_test)


In [29]:
X_train_imputed.shape

(298, 16)

In [30]:
X_val_imputed.shape

(38, 16)

In [31]:
y_train.dtypes

dtype('bool')

In [32]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(solver='lbfgs')
log_reg.fit(X_train_imputed, y_train)
print('Validation Accuracy', log_reg.score(X_val_imputed, y_val))

Validation Accuracy 0.7631578947368421


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [33]:
log_reg.intercept_

array([-31.66767518])

In [0]:
# The logistic sigmoid "squishing" function, implemented to accept numpy arrays
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.e**(-x))

In [35]:
sigmoid(log_reg.intercept_ + np.dot(log_reg.coef_, np.transpose(X_test_imputed)))

array([[0.99988703, 0.99672754, 0.00107603, 0.99996688, 0.00206823,
        0.14208153, 0.91248605, 0.99874831, 0.47344065, 0.92155703,
        0.60282292, 0.05789694, 0.88407515, 0.49836734, 0.86010712,
        0.9420349 , 0.74413641, 0.01289386, 0.00367465, 0.01124866,
        0.01154337, 0.97172609, 0.39033357, 0.68492377, 0.85542216,
        0.07695965, 0.05933486, 0.99995995, 0.95248744, 0.99075578,
        0.09786482, 0.74241122, 0.9568421 , 0.03290034, 0.75034058,
        0.7136496 , 0.99941138, 0.9640126 ]])

## Get your model's validation accuracy. (Multiple times if you try multiple iterations.)

In [36]:
print('Validation Accuracy', log_reg.score(X_val_imputed, y_val))

Validation Accuracy 0.7631578947368421


## Get your model's test accuracy. (One time, at the end.)

In [37]:
log_reg = LogisticRegression(solver='lbfgs')
log_reg.fit(X_train_imputed, y_train)
print('Models test Accuracy', log_reg.score(X_test_imputed, y_test))

Validation Accuracy 0.7631578947368421


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
